In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
import sys
from Bio import SeqIO

In [22]:
filename = "uniref50.fasta"
#aa1_to_index = {'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6,
#                    'I': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 11, 'P': 12,
#                    'Q': 13, 'R': 14, 'S': 15, 'T': 16, 'V': 17, 'W': 18,
#                    'Y': 19, 'X':20, 'NOSEQ':21, '-': 22}

temp_dict = {}
#aa1_to_index = {'A':0, 'B':1}

#print(aa1_to_index['B'])
print("Reading in data...", end='')
sys.stdout.flush()
seqs = []
counter = 0
count = 0
print(SeqIO.parse(filename, "fasta"))

num_seq = 100000

for record in SeqIO.parse(filename, "fasta"):
    #seqs.append(np.array([aa1_to_index[aa] for aa in str(record.seq).upper()]))
    for elem in str(record.seq).upper():
        if not (elem in temp_dict):
            temp_arr = np.zeros(25)
            temp_arr[len(temp_dict)] = 1
            temp_dict[elem] = temp_arr
    if (count > num_seq - 2):
        break
    else:
        count += 1

temp = []
for i in temp_dict:
    #print(i)
    temp.append(i)

temp.sort()
print(temp)

#seqs = np.random.choice(full_seqs, 100000, False)

'''
max_length = max([len(seq) for seq in seqs])

data = torch.empty((len(seqs), max_length))
for i, seq in enumerate(seqs):
    #print(seq)
    data[i, :] = torch.from_numpy(seq)
print("done")

print("Data shape: ", data.shape)

train_samples = int(len(data) * 0.8)
train_data = torch.utils.data.TensorDataset(data[:train_samples])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                               shuffle=True)
test_data = torch.utils.data.TensorDataset(data[train_samples:])
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                              shuffle=False)
'''

Reading in data...

<generator object FastaIterator at 0x7f7cedf88bf8>


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


'\nmax_length = max([len(seq) for seq in seqs])\n\ndata = torch.empty((len(seqs), max_length))\nfor i, seq in enumerate(seqs):\n    #print(seq)\n    data[i, :] = torch.from_numpy(seq)\nprint("done")\n\nprint("Data shape: ", data.shape)\n\ntrain_samples = int(len(data) * 0.8)\ntrain_data = torch.utils.data.TensorDataset(data[:train_samples])\ntrain_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,\n                                               shuffle=True)\ntest_data = torch.utils.data.TensorDataset(data[train_samples:])\ntest_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,\n                                              shuffle=False)\n'